In [1]:
pip install statsmodels

In [2]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

## statsmodels（先頭30フレームの平均）

In [3]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_30_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.996101  0.621411  0.310666  0.479798  0.311649  0.183596   
answer2     0.997114  0.620886  0.345670  0.512574  0.275988  0.183498   
answer3     0.995380  0.642572  0.413375  0.471471  0.396075  0.158026   
answer4     0.994671  0.690717  0.408082  0.437803  0.420779  0.149670   
answer5     0.994515  0.681902  0.402748  0.464709  0.434606  0.151975   
answer6     0.995428  0.686710  0.527085  0.380380  0.428165  0.162578   
answer7     0.994965  0.627859  0.411296  0.451731  0.489222  0.190342   
answer8     0.994378  0.681677  0.412118  0.554543  0.326517  0.161899   
answer9     0.996479  0.589496  0.367464  0.484712  0.404582  0.206949   
answer10    0.994109  0.695505  0.467914  0.500212  0.497803  0.179128   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.992400  0.789421  0.570049  0.411488  0.317958  0.229030   
answer2     0.989797  0.841541  0.676749  0.355240  0.321608  0.276907   
answer3     0.998660  0.876348  0.676918  0.284739  0.296028  0.421604   
answer4     0.998086  0.838945  0.745496  0.239638  0.339665  0.158786   
answer5     0.999250  0.857615  0.700484  0.218833  0.307734  0.316629   
answer7     0.998610  0.818492  0.648702  0.382689  0.268802  0.405825   
answer8     0.998368  0.874209  0.658484  0.272208  0.311818  0.363533   
answer9     0.999545  0.833738  0.737453  0.236162  0.313999  0.221221   
answer10    0.999570  0.829682  0.680429  0.270914  0.287688  0.375927   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                  
answer1   0.319731  0.471270  0.177268  0

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.998990  0.658023  0.525840  0.270455  0.401238  0.166355   
answer2     0.998917  0.592775  0.431903  0.315060  0.376275  0.197835   
answer3     0.998094  0.581915  0.425342  0.417832  0.412837  0.157647   
answer4     0.998058  0.512958  0.405950  0.291380  0.497986  0.109617   
answer5     0.997810  0.676783  0.506988  0.420932  0.383731  0.161912   
answer6     0.998376  0.591131  0.490080  0.402543  0.517737  0.121957   
answer7     0.998496  0.625088  0.475567  0.398995  0.376210  0.169292   
answer8     0.998343  0.556162  0.419704  0.423196  0.468877  0.260864   
answer9     0.999544  0.665609  0.588612  0.242838  0.413902  0.122578   
answer10    0.999172  0.654764  0.534927  0.331545  0.462546  0.160074   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [24]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2865
Time:                        00:12:26   Log-Likelihood:                -127.67
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 3.005e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.9657      2.736      2.180      0.029       0.603      11.328
AU01         -10.3232      2.573     -4.012      0.000     -15.367      -5.280
AU02           6.8949

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2923
Time:                        00:12:27   Log-Likelihood:                -126.63
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 1.232e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.7925      2.798      2.428      0.015       1.308      12.277
AU01          -9.3264      2.622     -3.556      0.000     -14.466      -4.187
AU02           6.6203      2.146      3.085      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.3448
Time:                        00:12:27   Log-Likelihood:                -117.24
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 3.469e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.5868      3.110      3.404      0.001       4.491      16.683
AU01         -12.2009      2.875     -4.243      0.000     -17.837      -6.565
AU02           7.6313      2.447      3.119      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2864
Time:                        00:12:27   Log-Likelihood:                -124.03
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 1.076e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.4194      2.761      2.325      0.020       1.009      11.830
AU01          -9.6572      2.603     -3.709      0.000     -14.760      -4.555
AU02           6.3387      2.175      2.915      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.3291
Time:                        00:12:27   Log-Likelihood:                -117.99
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 9.962e-17
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.2980      2.860      1.153      0.249      -2.308       8.904
AU01          -8.0037      2.735     -2.926      0.003     -13.364      -2.643
AU02           7.4392      2.284      3.256      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.3002
Time:                        00:12:27   Log-Likelihood:                -122.35
converged:                       True   LL-Null:                       -174.84
Covariance Type:            nonrobust   LLR p-value:                 1.054e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.2335      2.800      2.226      0.026       0.746      11.721
AU01         -10.2861      2.668     -3.855      0.000     -15.515      -5.057
AU02           7.2771      2.195      3.315      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2866
Time:                        00:12:28   Log-Likelihood:                -128.01
converged:                       True   LL-Null:                       -179.44
Covariance Type:            nonrobust   LLR p-value:                 2.620e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.0753      2.737      2.220      0.026       0.711      11.440
AU01         -10.0918      2.587     -3.901      0.000     -15.162      -5.022
AU02           6.7190      2.175      3.089      0.0

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4580966953035052, 0.44209326758347584, 0.3209633772014362, 0.3147915206736055, 0.401017370174393, 0.3188767121616385, 0.348964265319105, 0.48513742914176844, 0.4676602000069422, 0.4025695043751122]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.4091757753864867, 0.8052578976923651, 0.34565227073385985, 0.6228131882427576, 0.29434332441807076, 0.34932950608239177, 0.49409478768833015, 0.28132069709591245, 0.21961659281087598, 0.5469333833959416]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1,

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.31729539707766247, 0.22005868419580307, 0.3675184264368963, 0.3283217954504631, 0.33043491902572286, 0.3317166398344786, 0.2948353988583826, 0.3592351433382479]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3703621784832245, 0.3717813148565006, 0.3682188257300266, 0.3677508771334533, 0.36985387603786807, 0.36859354654276943, 0.36811972858151293, 0.3679177849688817]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.49517532572998785, 0.23054660902070098, 0.26931818491993026, 0.29162309469

## statsmodels（先頭10フレームの平均）

In [5]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_10_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.996606  0.601540  0.310669  0.446768  0.329936  0.174976   
answer2     0.997146  0.582202  0.349965  0.518763  0.280240  0.176288   
answer3     0.996037  0.586778  0.323077  0.512440  0.306640  0.165145   
answer4     0.994471  0.688244  0.389618  0.471758  0.387922  0.144351   
answer5     0.994645  0.645536  0.329584  0.537709  0.398903  0.168681   
answer6     0.995012  0.646885  0.477473  0.382850  0.379168  0.163435   
answer7     0.995345  0.616554  0.359053  0.489489  0.432771  0.202691   
answer8     0.994680  0.674456  0.370218  0.526292  0.357387  0.164901   
answer9     0.995906  0.629347  0.358147  0.483077  0.408082  0.176283   
answer10    0.993830  0.710357  0.484428  0.420916  0.505945  0.167669   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.996531  0.483951  0.442325  0.452735  0.398179  0.261880   
answer2     0.996840  0.548615  0.351192  0.565752  0.404251  0.209536   
answer3     0.997289  0.614440  0.426585  0.494938  0.343004  0.223382   
answer5     0.991386  0.467649  0.345694  0.518877  0.363556  0.366371   
answer6     0.990495  0.573666  0.393952  0.569486  0.330103  0.234928   
answer7     0.994884  0.470712  0.254099  0.591489  0.356063  0.262028   
answer8     0.994305  0.634717  0.419036  0.456793  0.376306  0.262135   
answer9     0.995187  0.615919  0.516444  0.504904  0.374154  0.332428   
answer10    0.987952  0.654090  0.572765  0.432125  0.350632  0.295209   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                  
answer1   0.265290  0.285492  0.242733  0

In [26]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2391
Time:                        00:12:35   Log-Likelihood:                -136.15
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 3.848e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.8159      2.414      0.752      0.452      -2.915       6.547
AU01          -6.8933      2.263     -3.046      0.002     -11.329      -2.458
AU02           4.9901

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2409
Time:                        00:12:35   Log-Likelihood:                -133.50
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 5.490e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1130      2.502      0.445      0.656      -3.791       6.017
AU01          -6.4769      2.296     -2.821      0.005     -10.977      -1.977
AU02           4.7069      1.883      2.499      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2488
Time:                        00:12:36   Log-Likelihood:                -134.42
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 9.098e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2491      2.499      0.900      0.368      -2.648       7.147
AU01          -6.4520      2.295     -2.811      0.005     -10.950      -1.954
AU02           4.5248      1.913      2.365      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2419
Time:                        00:12:36   Log-Likelihood:                -131.77
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 7.182e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.3839      2.714      1.984      0.047       0.064      10.704
AU01          -7.7909      2.369     -3.288      0.001     -12.435      -3.147
AU02           4.7069      1.925      2.445      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2441
Time:                        00:12:36   Log-Likelihood:                -131.38
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 5.199e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.6149      2.523      1.433      0.152      -1.330       8.560
AU01          -6.1221      2.284     -2.681      0.007     -10.598      -1.646
AU02           4.8603      1.893      2.568      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2891
Time:                        00:12:36   Log-Likelihood:                -123.57
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 7.245e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3746      2.558      0.146      0.884      -4.639       5.388
AU01          -3.9778      2.470     -1.610      0.107      -8.820       0.864
AU02           4.3975      1.987      2.213      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2506
Time:                        00:12:37   Log-Likelihood:                -134.10
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 6.993e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0962      2.477      1.250      0.211      -1.759       7.952
AU01          -6.8938      2.268     -3.039      0.002     -11.340      -2.448
AU02           5.1796      1.885      2.748      0.0

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4477507225785844, 0.42465319006721786, 0.44671347828024166, 0.35793816818819973, 0.4835122429451197, 0.3007399590984962, 0.42302199745373203, 0.4863962460847783, 0.4805170822103122, 0.358062188518453]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.39504992699207775, 0.8436984289260421, 0.3812647515994513, 0.5321679475804907, 0.3230414413157192, 0.30070179675690106, 0.3319578271788067, 0.2659437788550727, 0.24304233844222675, 0.5360131059735127]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.28698108509722176, 0.2197170707957278, 0.3105018697376564, 0.30551502142441417, 0.33985979334497013, 0.3191089034739449, 0.30954821784538533, 0.3366337206434636]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36955508670299014, 0.37236684782282303, 0.3683058956018876, 0.3696975801381851, 0.37136551395138795, 0.36915143396980943, 0.3680479503310403, 0.36842991439344674]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4730348974678777, 0.20095107816367747, 0.22363781153329867, 0.329337788

## statsmodels（先頭30フレームの標準偏差）

In [7]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001530  0.019816  0.015263  0.043458  0.039679  0.024692   
answer2     0.000945  0.036016  0.030624  0.016067  0.019143  0.011539   
answer3     0.001572  0.043948  0.069743  0.042619  0.075438  0.013640   
answer4     0.001297  0.023535  0.022897  0.034610  0.033839  0.018583   
answer5     0.001613  0.030034  0.058682  0.063061  0.046598  0.021786   
answer6     0.001492  0.035972  0.063001  0.025672  0.045928  0.013917   
answer7     0.001258  0.014414  0.046258  0.036940  0.063940  0.020970   
answer8     0.001798  0.013718  0.041780  0.050493  0.031532  0.015747   
answer9     0.001176  0.050766  0.018523  0.044329  0.028126  0.027568   
answer10    0.001767  0.019214  0.023940  0.072549  0.034567  0.012300   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001278  0.025778  0.027126  0.049983  0.019370  0.024750   
answer2     0.002824  0.046602  0.068367  0.049399  0.036165  0.034254   
answer3     0.003218  0.020322  0.055340  0.057530  0.033376  0.013081   
answer4     0.005926  0.069519  0.104422  0.068148  0.039862  0.012588   
answer5     0.003032  0.024849  0.031940  0.032309  0.042673  0.013353   
answer6     0.003835  0.072617  0.104583  0.073711  0.036295  0.026880   
answer7     0.002169  0.025682  0.074075  0.034124  0.026793  0.030346   
answer8     0.002700  0.014664  0.035277  0.034672  0.012480  0.017666   
answer9     0.003261  0.047805  0.048925  0.085012  0.016441  0.020873   
answer10    0.001270  0.022781  0.037086  0.041994  0.014544  0.018152   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [28]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1183
Time:                        00:12:45   Log-Likelihood:                -157.77
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 0.0006009
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4752      0.452      1.051      0.293      -0.411       1.361
AU01          -3.8175      8.320     -0.459      0.646     -20.124      12.489
AU02           5.5394

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1089
Time:                        00:12:45   Log-Likelihood:                -156.70
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                  0.002226
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2263      0.461      0.491      0.623      -0.677       1.130
AU01          -4.4609      8.347     -0.534      0.593     -20.820      11.898
AU02           6.5446      6.975      0.938      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1212
Time:                        00:12:45   Log-Likelihood:                -157.25
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 0.0004216
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4283      0.454      0.943      0.346      -0.462       1.318
AU01          -0.7358      8.494     -0.087      0.931     -17.385      15.913
AU02           3.4586      7.252      0.477      0.6

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1219
Time:                        00:12:45   Log-Likelihood:                -157.57
converged:                       True   LL-Null:                       -179.44
Covariance Type:            nonrobust   LLR p-value:                 0.0003747
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2817      0.465      0.606      0.545      -0.630       1.193
AU01          -4.6345      8.371     -0.554      0.580     -21.041      11.772
AU02           3.1701      7.150      0.443      0.6

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1250
Time:                        00:12:45   Log-Likelihood:                -152.08
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 0.0004118
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3084      0.468      0.659      0.510      -0.609       1.226
AU01          -2.6760      8.513     -0.314      0.753     -19.361      14.009
AU02           1.4958      7.381      0.203      0.8

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1227
Time:                        00:12:46   Log-Likelihood:                -154.29
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 0.0004562
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3380      0.464      0.729      0.466      -0.571       1.247
AU01          -2.2201      8.383     -0.265      0.791     -18.650      14.210
AU02           6.7013      7.076      0.947      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      252
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1223
Time:                        00:12:46   Log-Likelihood:                -158.36
converged:                       True   LL-Null:                       -180.43
Covariance Type:            nonrobust   LLR p-value:                 0.0003265
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4581      0.459      0.997      0.319      -0.442       1.358
AU01          -1.8717      8.266     -0.226      0.821     -18.072      14.329
AU02           4.6116      7.046      0.655      0.5

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3934695302282269, 0.3933333806161451, 0.3925098536796478, 0.3930472676076984, 0.3933537007390909, 0.39325778907856107, 0.39304517410162293, 0.39347895774649283, 0.3930979420792343, 0.39351271766197277]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.39371323375961886, 0.39358963107679973, 0.3933394513456098, 0.39354666184946463, 0.3932771596641497, 0.3931757897711767, 0.3928191102424839, 0.39345151726235, 0.39325433114190533, 0.39339463706989697]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1

ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36046456827120116, 0.3609481664928512, 0.3607208225875555, 0.3606462472063252, 0.3608042713709542, 0.3604958341525813, 0.36044884842379893, 0.36045791521339643]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36070015933543126, 0.36094816595139934, 0.36071055099998905, 0.3608132794423222, 0.3609147352571504, 0.3607443300872828, 0.36098222014364806, 0.36092993079729824]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35593173662084293, 0.35562686791274933, 0.35506284019141304, 0.3551807578537661, 0.35606912834943244, 0.3557917162004538, 0

## statsmodels（先頭10フレームの標準偏差）

In [21]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001006  0.009949  0.010251  0.015674  0.021276  0.007454   
answer2     0.001069  0.013489  0.030429  0.011434  0.019826  0.014543   
answer3     0.001248  0.020389  0.027520  0.029420  0.003884  0.013260   
answer4     0.001489  0.015146  0.014760  0.034909  0.011303  0.006671   
answer5     0.001968  0.007569  0.016442  0.058506  0.037077  0.015832   
answer6     0.001509  0.020592  0.051012  0.026717  0.028588  0.010255   
answer7     0.001398  0.016808  0.018877  0.007412  0.020422  0.017243   
answer8     0.001651  0.007443  0.022618  0.010983  0.004949  0.021055   
answer9     0.000932  0.042140  0.010872  0.033027  0.035538  0.006528   
answer10    0.002075  0.021212  0.011367  0.018154  0.023781  0.003617   

              AU09      AU10      AU12      AU14      AU15      AU17  \

In [22]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1609
Time:                        02:45:40   Log-Likelihood:                -150.14
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 2.610e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9333      0.468     -1.993      0.046      -1.851      -0.015
AU01          -1.8661     12.418     -0.150      0.881     -26.204      22.472
AU02           6.9106

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1565
Time:                        02:45:40   Log-Likelihood:                -150.94
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 4.729e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0934      0.465     -2.352      0.019      -2.005      -0.182
AU01          -4.5035     12.621     -0.357      0.721     -29.239      20.232
AU02           9.6771      9.537      1.015      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1537
Time:                        02:45:41   Log-Likelihood:                -151.44
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 6.842e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0737      0.469     -2.291      0.022      -1.992      -0.155
AU01          -2.8987     12.336     -0.235      0.814     -27.077      21.280
AU02           9.3839      9.237      1.016      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1778
Time:                        02:45:41   Log-Likelihood:                -142.90
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 5.260e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0576      0.481     -2.197      0.028      -2.001      -0.114
AU01           0.5818     13.026      0.045      0.964     -24.948      26.112
AU02           3.5000      9.871      0.355      0.7

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      252
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1668
Time:                        02:45:42   Log-Likelihood:                -150.34
converged:                       True   LL-Null:                       -180.43
Covariance Type:            nonrobust   LLR p-value:                 9.820e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9940      0.463     -2.146      0.032      -1.902      -0.086
AU01          -2.1629     12.358     -0.175      0.861     -26.385      22.059
AU02           7.5239      9.214      0.817      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      252
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1552
Time:                        02:45:42   Log-Likelihood:                -152.42
converged:                       True   LL-Null:                       -180.43
Covariance Type:            nonrobust   LLR p-value:                 4.713e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0801      0.463     -2.332      0.020      -1.988      -0.172
AU01          -0.5808     12.178     -0.048      0.962     -24.449      23.287
AU02           6.9075      9.056      0.763      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Wed, 08 Jan 2025   Pseudo R-squ.:                  0.1716
Time:                        02:45:42   Log-Likelihood:                -148.65
converged:                       True   LL-Null:                       -179.44
Covariance Type:            nonrobust   LLR p-value:                 5.770e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9905      0.470     -2.110      0.035      -1.911      -0.070
AU01           5.0035     12.538      0.399      0.690     -19.571      29.578
AU02           5.3167      9.174      0.580      0.5

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35590811229903385, 0.3909288879470018, 0.3208384611773205, 0.37141806597206367, 0.4143948389967777, 0.3854964220779338, 0.37857554243232366, 0.44308965410629997, 0.3788412681882331, 0.361343002278338]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.34780218551824155, 0.4208850368440902, 0.40905489208561263, 0.3552957607742417, 0.4361353972448098, 0.38053879492011117, 0.4158410331080171, 0.37102733057036935, 0.3559313717135949, 0.37475300108778836]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1,

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36070042372918004, 0.3606376518855688, 0.36087813667022123, 0.3607157424658295, 0.36072691963667175, 0.36058640071750464, 0.3606488485580524, 0.3606925888218626]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4524645185681812, 0.39647837051444834, 0.3946449342687275, 0.41105263482745363, 0.41075627087354083, 0.39277866731367783, 0.39317300451050524, 0.42602125018583603]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.40425870777800693, 0.3460752755556537, 0.3441403455401902, 0.3571196

## statsmodels（先頭30フレームの差分）

In [3]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 結果をリストに保存
    result_list.append((file_name, diff_values_per_label))

    # Yを追加
    diff_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
        label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     answer1    0.002523 -0.001195  0.014545  0.005055  0.002442  0.002156   
2     answer1   -0.000777 -0.000953  0.005555 -0.014980  0.017999 -0.007861   
3     answer1   -0.000558 -0.015196 -0.016889  0.003133 -0.018193 -0.002822   
4     answer1    0.000219  0.006084 -0.002855 -0.008630 -0.000543  0.006542   
5     answer1    0.000912 -0.002755 -0.010911  0.001786 -0.009210  0.003166   
..        ...         ...       ...       ...       ...       ...       ...   
295  answer10    0.004307  0.001851 -0.008759  0.021084 -0.012385 -0.002022   
296  answer10   -0.000085  0.008260  0.028112 -0.026262 -0.018536 -0.004130   
297  answer10   -0.003561  0.014570  0.006747 -0.037738 -0.005984 -0.002483   
298  answer10    0.001567  0.006688  0.013547  0.007463 -0.011259  0.004897   
299  answer10    0.001315  0.003890  0.008731 -0.012653 -0.004096 -0.002768   

         AU

       label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1    answer2    0.019649  0.037536  0.003369  0.017613  0.001962 -0.047592   
2    answer2    0.011441  0.014443 -0.023149  0.020847  0.013525 -0.059588   
3    answer2    0.030803  0.005987  0.010621 -0.035973 -0.017580 -0.013268   
4    answer2   -0.006332  0.020538  0.035275 -0.033117  0.003294 -0.007449   
5    answer2    0.014331  0.013971 -0.011411 -0.008102 -0.002103 -0.058272   
..       ...         ...       ...       ...       ...       ...       ...   
211  answer9    0.000694  0.005001  0.017775 -0.011629  0.000739 -0.000971   
212  answer9    0.008434  0.006099 -0.000796  0.000908 -0.002027 -0.000409   
213  answer9    0.003799 -0.015477 -0.030996  0.014215 -0.004243 -0.000389   
214  answer9   -0.012034 -0.000040 -0.000572 -0.014941  0.000291  0.000773   
215  answer9    0.009388  0.004352 -0.015454  0.043339 -0.003552  0.000193   

         AU09      AU10      AU12      AU14      AU15      AU17

In [30]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7120
Model:                          Logit   Df Residuals:                     7102
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                0.001811
Time:                        00:12:57   Log-Likelihood:                -4754.3
converged:                       True   LL-Null:                       -4762.9
Covariance Type:            nonrobust   LLR p-value:                    0.4377
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4434      0.024    -18.188      0.000      -0.491      -0.396
AU01           1.9312      2.307      0.837      0.403      -2.591       6.454
AU02          -0.4371

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7198
Model:                          Logit   Df Residuals:                     7180
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                0.001720
Time:                        00:12:57   Log-Likelihood:                -4681.7
converged:                       True   LL-Null:                       -4689.7
Covariance Type:            nonrobust   LLR p-value:                    0.5146
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5881      0.025    -23.814      0.000      -0.637      -0.540
AU01           1.4630      2.340      0.625      0.532      -3.123       6.049
AU02          -1.2219      1.899     -0.644      0.5

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7149
Model:                          Logit   Df Residuals:                     7131
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                0.001935
Time:                        00:12:58   Log-Likelihood:                -4768.0
converged:                       True   LL-Null:                       -4777.2
Covariance Type:            nonrobust   LLR p-value:                    0.3587
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4497      0.024    -18.468      0.000      -0.497      -0.402
AU01           1.4448      2.286      0.632      0.527      -3.035       5.924
AU02          -0.2022      1.862     -0.109      0.9

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7160
Model:                          Logit   Df Residuals:                     7142
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                0.001966
Time:                        00:12:58   Log-Likelihood:                -4773.2
converged:                       True   LL-Null:                       -4782.6
Covariance Type:            nonrobust   LLR p-value:                    0.3401
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4520      0.024    -18.570      0.000      -0.500      -0.404
AU01           2.3389      2.296      1.019      0.308      -2.161       6.839
AU02          -0.4905      1.878     -0.261      0.7

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7133
Model:                          Logit   Df Residuals:                     7115
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                0.001947
Time:                        00:12:58   Log-Likelihood:                -4613.2
converged:                       True   LL-Null:                       -4622.2
Covariance Type:            nonrobust   LLR p-value:                    0.3891
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6138      0.025    -24.653      0.000      -0.663      -0.565
AU01           2.2729      2.326      0.977      0.329      -2.286       6.832
AU02          -0.9883      1.893     -0.522      0.6

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7159
Model:                          Logit   Df Residuals:                     7141
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                0.002073
Time:                        00:12:59   Log-Likelihood:                -4639.7
converged:                       True   LL-Null:                       -4649.4
Covariance Type:            nonrobust   LLR p-value:                    0.3130
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6032      0.025    -24.308      0.000      -0.652      -0.555
AU01           2.9612      2.355      1.257      0.209      -1.654       7.577
AU02          -0.7452      1.933     -0.386      0.7

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7236
Model:                          Logit   Df Residuals:                     7218
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                0.002221
Time:                        00:12:59   Log-Likelihood:                -4809.0
converged:                       True   LL-Null:                       -4819.7
Covariance Type:            nonrobust   LLR p-value:                    0.2087
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4692      0.024    -19.340      0.000      -0.517      -0.422
AU01           2.5025      2.306      1.085      0.278      -2.017       7.022
AU02          -0.8284      1.878     -0.441      0.6

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7202
Model:                          Logit   Df Residuals:                     7184
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                0.001986
Time:                        00:12:59   Log-Likelihood:                -4684.5
converged:                       True   LL-Null:                       -4693.9
Covariance Type:            nonrobust   LLR p-value:                    0.3495
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5864      0.025    -23.759      0.000      -0.635      -0.538
AU01           1.8464      2.349      0.786      0.432      -2.758       6.451
AU02          -1.4267      1.906     -0.748      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7169
Model:                          Logit   Df Residuals:                     7151
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                0.001609
Time:                        00:13:00   Log-Likelihood:                -4652.3
converged:                       True   LL-Null:                       -4659.8
Covariance Type:            nonrobust   LLR p-value:                    0.5960
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5991      0.025    -24.168      0.000      -0.648      -0.551
AU01           1.6833      2.363      0.712      0.476      -2.948       6.315
AU02          -0.6638      1.925     -0.345      0.7

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                 7125
Model:                          Logit   Df Residuals:                     7107
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                0.002003
Time:                        00:13:00   Log-Likelihood:                -4755.8
converged:                       True   LL-Null:                       -4765.4
Covariance Type:            nonrobust   LLR p-value:                    0.3235
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4448      0.024    -18.253      0.000      -0.493      -0.397
AU01           1.4033      2.320      0.605      0.545      -3.144       5.951
AU02          -1.0286      1.876     -0.548      0.5

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID34_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID36_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0,

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID38_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID69_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0,

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID73_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## statsmodels（先頭30フレームの差分の標準偏差）

In [9]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 差分の標準偏差を計算
    std_values_per_label = (
        diff_values_per_label.groupby("label").std(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001462  0.009580  0.012407  0.016297  0.011874  0.007615   
answer2     0.001210  0.012272  0.015193  0.015429  0.009401  0.005542   
answer3     0.001548  0.010425  0.014770  0.011224  0.011701  0.004530   
answer4     0.001596  0.011475  0.010987  0.016799  0.017746  0.005220   
answer5     0.002208  0.008542  0.013593  0.017499  0.017769  0.006108   
answer6     0.001575  0.009464  0.016398  0.014048  0.016414  0.005451   
answer7     0.001426  0.008086  0.011682  0.019850  0.022139  0.005849   
answer8     0.001579  0.008420  0.013003  0.012297  0.006954  0.006512   
answer9     0.001348  0.013357  0.014014  0.018175  0.010446  0.004908   
answer10    0.002133  0.009996  0.012292  0.017410  0.010149  0.003508   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1          NaN       NaN       NaN       NaN       NaN       NaN   
answer2     0.001489  0.007765  0.012411  0.021777  0.004289  0.003467   
answer3     0.000420  0.008880  0.010858  0.016399  0.003883  0.002913   
answer4     0.000628  0.011773  0.011558  0.020367  0.006988  0.009850   
answer5          NaN       NaN       NaN       NaN       NaN       NaN   
answer6     0.019767  0.012390  0.025108  0.034852  0.005774  0.015906   
answer7     0.000108  0.009293  0.012002  0.012152  0.015235  0.002877   
answer8     0.004102  0.008301  0.019756  0.021228  0.005763  0.013388   
answer9     0.000066  0.006051  0.006710  0.012970  0.001475  0.003877   
answer10    0.000099  0.009661  0.010732  0.015345  0.009272  0.003626   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [32]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2125
Time:                        00:13:08   Log-Likelihood:                -140.92
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 1.920e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0867      0.800     -3.859      0.000      -4.654      -1.519
AU01         -19.7051     37.468     -0.526      0.599     -93.141      53.731
AU02          13.9999

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2116
Time:                        00:13:09   Log-Likelihood:                -141.08
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 2.192e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1129      0.806     -3.861      0.000      -4.693      -1.533
AU01         -25.1389     38.122     -0.659      0.510     -99.857      49.579
AU02          20.9815     39.640      0.529      0.5

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2433
Time:                        00:13:09   Log-Likelihood:                -135.40
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 2.068e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.8738      0.829     -3.469      0.001      -4.498      -1.250
AU01         -24.7104     38.846     -0.636      0.525    -100.847      51.426
AU02          11.2788     40.878      0.276      0.7

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2318
Time:                        00:13:09   Log-Likelihood:                -138.23
converged:                       True   LL-Null:                       -179.93
Covariance Type:            nonrobust   LLR p-value:                 9.475e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.2407      0.805     -4.024      0.000      -4.819      -1.662
AU01         -30.6321     38.240     -0.801      0.423    -105.580      44.316
AU02          20.0972     39.860      0.504      0.6

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  271
Model:                          Logit   Df Residuals:                      253
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2472
Time:                        00:13:10   Log-Likelihood:                -136.19
converged:                       True   LL-Null:                       -180.92
Covariance Type:            nonrobust   LLR p-value:                 7.641e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.9487      0.874     -4.519      0.000      -5.661      -2.236
AU01         -23.0215     38.392     -0.600      0.549     -98.269      52.226
AU02           8.1910     40.099      0.204      0.8

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2351
Time:                        00:13:10   Log-Likelihood:                -132.94
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 1.882e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.7629      0.851     -3.248      0.001      -4.430      -1.096
AU01         -78.6036     43.517     -1.806      0.071    -163.896       6.689
AU02          33.5266     41.562      0.807      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2371
Time:                        00:13:10   Log-Likelihood:                -136.51
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 5.186e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0146      0.831     -3.628      0.000      -4.643      -1.386
AU01         -12.0684     38.050     -0.317      0.751     -86.646      62.509
AU02          29.7013     40.808      0.728      0.4

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3449326510680279, 0.359670815790138, 0.2841733164460852, 0.2692760687909498, 0.3053379921266193, 0.2903692028369379, 0.30687137108127877, 0.35331322028004214, 0.32015732884479575, 0.2846162651971748]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3219482303074483, 0.5009055328453627, 0.2888427362518468, 0.514400516651315, 0.3220054049679685, 0.27482706051963895, 0.3497219551118393, 0.28406383259614343, 0.26854596524700575, 0.34714132106389717]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1

最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 25.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 1, 0, 0, 1, 0, 0]
クラス1の確率: [0.3841156558469006, 0.43312553302768825, 0.6600487682137305, 0.4520270331320724, 0.40185992204118226, 0.6137417977762744, 0.4661424487076254, 0.46515390753998836]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 25.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 1, 1, 0, 0, 0]
クラス1の確率: [0.48491068256728936, 0.45239302081927474, 0.39436058699934634, 0.504115914930513, 0.5456738250477287, 0.4139634638688885, 0.36848846333025076, 0.3554335513076562]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35583594225811455, 0.3557738040891075, 0.3558413032914614, 0.3558088

## statsmodels（先頭30フレームの最大値）

In [11]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに30フレームの最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.998552  0.659222  0.342934  0.560316  0.386278  0.225416   
answer2     0.998392  0.687580  0.381302  0.549067  0.308328  0.201725   
answer3     0.998105  0.697356  0.488168  0.556769  0.494694  0.183472   
answer4     0.996902  0.730949  0.459202  0.527441  0.481439  0.189705   
answer5     0.997551  0.717898  0.484356  0.631736  0.521545  0.198245   
answer6     0.997832  0.741541  0.631410  0.431417  0.504093  0.194596   
answer7     0.997573  0.657243  0.472553  0.499724  0.594109  0.225480   
answer8     0.997223  0.704030  0.483926  0.640750  0.367383  0.199509   
answer9     0.998192  0.686589  0.426857  0.570879  0.444240  0.246305   
answer10    0.996892  0.746445  0.505533  0.631177  0.537849  0.197410   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.998638  0.444472  0.259897  0.827659  0.320248  0.180905   
answer2     0.999302  0.418535  0.324947  0.780637  0.416125  0.077909   
answer3     0.999002  0.687466  0.651939  0.843843  0.453722  0.173935   
answer4     0.999044  0.738771  0.644856  0.758969  0.391232  0.096545   
answer5     0.998381  0.462345  0.393583  0.629510  0.405388  0.136519   
answer6     0.998488  0.597600  0.537364  0.739793  0.420776  0.154222   
answer7     0.999086  0.533943  0.357420  0.634649  0.355742  0.114601   
answer8     0.998767  0.460284  0.349113  0.588089  0.306919  0.099806   
answer9     0.999179  0.519601  0.407638  0.746318  0.344399  0.086832   
answer10    0.998726  0.574917  0.456134  0.644051  0.377775  0.157120   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [34]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2131
Time:                        00:13:18   Log-Likelihood:                -140.80
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 1.744e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.5459      2.322      2.388      0.017       0.995      10.097
AU01          -6.3683      2.242     -2.841      0.005     -10.762      -1.974
AU02           4.5663

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2186
Time:                        00:13:18   Log-Likelihood:                -137.42
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 1.369e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.0936      2.456      2.074      0.038       0.281       9.906
AU01          -6.5796      2.315     -2.842      0.004     -11.116      -2.043
AU02           4.8384      1.714      2.823      0.0

ファイル名: filtered_ID38_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 10/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2260
Time:                        00:13:18   Log-Likelihood:                -138.51
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 2.692e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.7475      2.413      2.796      0.005 

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2349
Time:                        00:13:19   Log-Likelihood:                -137.67
converged:                       True   LL-Null:                       -179.93
Covariance Type:            nonrobust   LLR p-value:                 5.940e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.6676      2.387      2.794      0.005       1.990      11.345
AU01          -6.6272      2.340     -2.832      0.005     -11.214      -2.041
AU02           4.6273      1.765      2.622      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  271
Model:                          Logit   Df Residuals:                      253
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2351
Time:                        00:13:19   Log-Likelihood:                -138.38
converged:                       True   LL-Null:                       -180.92
Covariance Type:            nonrobust   LLR p-value:                 4.755e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.0583      2.330      2.600      0.009       1.491      10.625
AU01          -7.0911      2.294     -3.092      0.002     -11.587      -2.596
AU02           5.2304      1.709      3.061      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2434
Time:                        00:13:19   Log-Likelihood:                -131.51
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 5.796e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.3315      2.435      2.189      0.029       0.558      10.105
AU01          -4.5918      2.409     -1.906      0.057      -9.313       0.129
AU02           4.3788      1.755      2.495      0.0

ファイル名: filtered_ID69_smoothed.csv
予測率: 85.71%
正解ラベル: [0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 1, 0]

===== テストデータ: ファイル 28/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      252
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2082
Time:                        00:13:19   Log-Likelihood:                -140.05
converged:                       True   LL-Null:                       -176.88
Covariance Type:            nonrobust   LLR p-value:                 5.016e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.4003      2.365      2.706      0.007       1.765      11

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4002347952480501, 0.39970595275761533, 0.38514891185138184, 0.3863503210760913, 0.3963152467668054, 0.3892681422145575, 0.3857472388307233, 0.4003123659713522, 0.38995639580238006, 0.39430070792038185]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.41310863093424616, 0.74896859933551, 0.40864408978178984, 0.6870675635618594, 0.31005477491661193, 0.33202436178096756, 0.3422192558652844, 0.33682974936316834, 0.22813091003055774, 0.5874696967500835]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3018969286030019, 0.23175307195069092, 0.3437857729933216, 0.3222523026826209, 0.32160055382655245, 0.2871476878728583, 0.22088461107145202, 0.34679875515516495]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.37122738934512894, 0.3734950861753902, 0.37091564653212156, 0.3689744661983322, 0.37236834878350017, 0.36938299209319847, 0.3705379805624336, 0.36928558111392545]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.45595288027660136, 0.1696576044502725, 0.2673756335341508, 0.2828250314

## statsmodels（先頭10フレームの最大値）

In [13]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに10フレームの最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.998552  0.614411  0.325737  0.466249  0.362126  0.192537   
answer2     0.998392  0.611207  0.381302  0.533734  0.308328  0.201725   
answer3     0.998105  0.623145  0.365551  0.556769  0.311882  0.183472   
answer4     0.996902  0.713736  0.410865  0.527441  0.405043  0.151519   
answer5     0.996959  0.660467  0.352912  0.631736  0.443868  0.198245   
answer6     0.997096  0.684254  0.561657  0.431417  0.409441  0.184528   
answer7     0.997573  0.639601  0.378727  0.499724  0.471975  0.225345   
answer8     0.997107  0.685622  0.416294  0.541601  0.367383  0.199509   
answer9     0.996756  0.686589  0.374178  0.536184  0.444240  0.188060   
answer10    0.996236  0.746445  0.505533  0.444868  0.537849  0.172488   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.991801  0.494396  0.323083  0.746633  0.398570  0.217909   
answer2     0.974695  0.674752  0.616570  0.428085  0.602442  0.242415   
answer3     0.983904  0.625846  0.511488  0.499713  0.432027  0.335939   
answer4     0.986775  0.645236  0.597200  0.430825  0.600338  0.251927   
answer5     0.966149  0.570539  0.526615  0.376516  0.519433  0.374768   
answer6     0.978159  0.676978  0.671673  0.369179  0.506968  0.285296   
answer7     0.992321  0.594631  0.460918  0.465976  0.507627  0.384392   
answer8     0.964559  0.652105  0.597463  0.417793  0.611848  0.413031   
answer9     0.976772  0.633411  0.577865  0.456445  0.697429  0.096248   
answer10    0.969103  0.588956  0.540722  0.342688  0.582087  0.296104   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [36]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2228
Time:                        00:13:27   Log-Likelihood:                -139.08
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 4.304e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5191      2.274      0.228      0.819      -3.939       4.977
AU01          -5.4714      2.219     -2.466      0.014      -9.820      -1.123
AU02           4.7741

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2187
Time:                        00:13:27   Log-Likelihood:                -137.41
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 1.349e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4358      2.374     -0.184      0.854      -5.089       4.217
AU01          -4.8552      2.221     -2.187      0.029      -9.207      -0.503
AU02           4.7121      1.704      2.766      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2182
Time:                        00:13:28   Log-Likelihood:                -139.89
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 8.312e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8592      2.283      0.376      0.707      -3.615       5.334
AU01          -5.3658      2.230     -2.406      0.016      -9.737      -0.995
AU02           4.1141      1.775      2.318      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2267
Time:                        00:13:28   Log-Likelihood:                -134.40
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 6.233e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.6778      2.491      1.476      0.140      -1.205       8.561
AU01          -6.1445      2.294     -2.679      0.007     -10.640      -1.649
AU02           4.5416      1.763      2.576      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2549
Time:                        00:13:28   Log-Likelihood:                -133.70
converged:                       True   LL-Null:                       -179.44
Covariance Type:            nonrobust   LLR p-value:                 3.289e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1065      2.313     -0.046      0.963      -4.640       4.427
AU01          -5.1094      2.214     -2.307      0.021      -9.450      -0.769
AU02           5.4330      1.752      3.100      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2657
Time:                        00:13:28   Log-Likelihood:                -129.14
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 1.441e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5784      2.419     -0.653      0.514      -6.319       3.163
AU01          -2.5193      2.351     -1.072      0.284      -7.127       2.089
AU02           4.9061      1.774      2.765      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2312
Time:                        00:13:29   Log-Likelihood:                -134.42
converged:                       True   LL-Null:                       -174.84
Covariance Type:            nonrobust   LLR p-value:                 2.713e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9078      2.302      0.394      0.693      -3.603       5.419
AU01          -5.0373      2.245     -2.243      0.025      -9.438      -0.636
AU02           5.1295      1.736      2.954      0.0

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.2816
Time:                        00:13:29   Log-Likelihood:                -129.26
converged:                       True   LL-Null:                       -179.93
Covariance Type:            nonrobust   LLR p-value:                 4.989e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0511      2.433      0.843      0.399      -2.718       6.820
AU01          -8.5346      2.477     -3.445      0.001     -13.390      -3.679
AU02           7.3027      1.893      3.858      0.0

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
クラス1の確率: [0.4605464128461021, 0.44105234474769756, 0.4150825498859756, 0.36316268356199194, 0.5390261422775346, 0.2771343778604195, 0.4243411851163344, 0.5290471105546916, 0.4888164692154399, 0.3303293278169846]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3514697323905407, 0.8597404313355897, 0.36323609181579397, 0.5183699387016115, 0.29610357274624255, 0.28110404574170356, 0.3212671919001276, 0.2749788994294776, 0.2007791241245742, 0.49960890037282984]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1

最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.29370261523119334, 0.21390530950859984, 0.3738588696732816, 0.3141881853390211, 0.33407796179328925, 0.280833066065297, 0.3128895856929444, 0.3241696013164833]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3712258298781873, 0.3735057377094254, 0.36900248358928145, 0.37019292042826724, 0.3722878605923849, 0.36995810539332513, 0.3680722335930584, 0.368828720155117]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4263285867151961, 0.19262574213502062, 0.2319836901103533, 0.290125824930047

## statsmodels（先頭30フレームの最大値と最小値の差分）

In [15]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_30_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.006316  0.075649  0.063028  0.145261  0.127436  0.088201   
answer2     0.003334  0.118679  0.108644  0.066483  0.071918  0.044361   
answer3     0.005948  0.133020  0.202349  0.147331  0.195239  0.046483   
answer4     0.004306  0.070528  0.087552  0.145972  0.120498  0.066656   
answer5     0.006226  0.080696  0.175598  0.229915  0.178028  0.080477   
answer6     0.005584  0.119441  0.215140  0.099514  0.172244  0.046470   
answer7     0.005535  0.066280  0.144592  0.132392  0.192303  0.063128   
answer8     0.006327  0.042552  0.142992  0.147651  0.081356  0.069137   
answer9     0.004578  0.170315  0.085489  0.146519  0.095403  0.078796   
answer10    0.006379  0.077472  0.080922  0.231260  0.119278  0.035864   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1          NaN       NaN       NaN       NaN       NaN       NaN   
answer2     0.002378  0.041282  0.067534  0.141507  0.132828  0.031358   
answer3     0.001793  0.032982  0.057603  0.269593  0.080077  0.026433   
answer4     0.003755  0.062191  0.113327  0.135757  0.195827  0.024095   
answer5     0.002852  0.103351  0.039890  0.108216  0.098946  0.034246   
answer6          NaN       NaN       NaN       NaN       NaN       NaN   
answer7     0.005499  0.220395  0.233853  0.192436  0.070274  0.036617   
answer8          NaN       NaN       NaN       NaN       NaN       NaN   
answer9          NaN       NaN       NaN       NaN       NaN       NaN   
answer10    0.001057  0.045168  0.086826  0.062604  0.057468  0.029105   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.002719  0.051958  0.154974  0.095664  0.144231  0.109545   
answer2     0.002235  0.142535  0.209510  0.126172  0.135279  0.050433   
answer3     0.005324  0.181284  0.120317  0.192981  0.165154  0.119891   
answer4     0.004595  0.321962  0.335953  0.159153  0.140130  0.023505   
answer5     0.004819  0.115303  0.219258  0.302628  0.054249  0.161633   
answer6     0.002455  0.353299  0.404176  0.149859  0.123732  0.059489   
answer7     0.001795  0.191585  0.193123  0.352199  0.176883  0.174376   
answer8     0.002269  0.205031  0.183783  0.158895  0.089330  0.146878   
answer9     0.001010  0.028326  0.058965  0.118903  0.102312  0.017535   
answer10    0.001783  0.067435  0.147385  0.230920  0.061901  0.069186   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [38]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1368
Time:                        00:13:37   Log-Likelihood:                -154.46
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 6.119e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1001      0.486      0.206      0.837      -0.853       1.054
AU01          -2.4585      2.725     -0.902      0.367      -7.800       2.883
AU02           1.7456

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1317
Time:                        00:13:37   Log-Likelihood:                -152.70
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 0.0001540
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1689      0.497     -0.340      0.734      -1.144       0.806
AU01          -2.5669      2.737     -0.938      0.348      -7.931       2.797
AU02           1.9732      2.449      0.806      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1427
Time:                        00:13:37   Log-Likelihood:                -153.41
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 2.890e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0676      0.489      0.138      0.890      -0.890       1.026
AU01          -1.5314      2.777     -0.552      0.581      -6.973       3.911
AU02           1.0546      2.525      0.418      0.6

ファイル名: filtered_ID41_smoothed.csv
予測率: 60.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 1, 1, 1, 0, 1, 0, 0]

===== テストデータ: ファイル 13/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1430
Time:                        00:13:38   Log-Likelihood:                -153.77
converged:                       True   LL-Null:                       -179.44
Covariance Type:            nonrobust   LLR p-value:                 2.619e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0927      0.498     -0.186      0.852  

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  271
Model:                          Logit   Df Residuals:                      253
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1430
Time:                        00:13:38   Log-Likelihood:                -155.04
converged:                       True   LL-Null:                       -180.92
Covariance Type:            nonrobust   LLR p-value:                 2.250e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0900      0.488      0.184      0.854      -0.867       1.047
AU01          -2.3699      2.710     -0.875      0.382      -7.681       2.941
AU02           1.9834      2.425      0.818      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1641
Time:                        00:13:38   Log-Likelihood:                -145.28
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 3.198e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3649      0.527      0.692      0.489      -0.668       1.398
AU01          -6.3776      3.105     -2.054      0.040     -12.463      -0.292
AU02           1.9933      2.569      0.776      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      252
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1426
Time:                        00:13:39   Log-Likelihood:                -151.65
converged:                       True   LL-Null:                       -176.88
Covariance Type:            nonrobust   LLR p-value:                 3.592e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1571      0.495      0.317      0.751      -0.813       1.127
AU01          -1.4301      2.740     -0.522      0.602      -6.800       3.940
AU02           1.6804      2.453      0.685      0.4

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3938394251920313, 0.3938104787135678, 0.38827243239578263, 0.39125247928769297, 0.39341609869068694, 0.39270554357180204, 0.3909911214461155, 0.3946644888644296, 0.39149840534749797, 0.3941892540715777]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.395841117040652, 0.3957957192218269, 0.3931286892770825, 0.396234673640126, 0.3929956649124135, 0.39223374598871996, 0.3903398975414742, 0.39371558158357495, 0.39231746103950355, 0.3942908538803992]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1,

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3611290087473655, 0.36299941199957136, 0.3619762359904739, 0.36234554158229837, 0.362761090656233, 0.3615588263484092, 0.3635710676465555, 0.362947317623593]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3566650064959662, 0.35379433418871664, 0.35163221286262436, 0.3521916921523991, 0.35736441091704946, 0.35503344534662384, 0.35259325081385984, 0.3582796727533456, 0.351869765862418, 0.3524599638345634]

===== テストデータ: ファイル 25/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID63_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35166197750234635, 0.357

## xgboost（先頭10フレームの最大値と最小値の差分）

In [17]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_10_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.003534  0.030838  0.030655  0.051194  0.063530  0.027755   
answer2     0.003334  0.042307  0.099821  0.030257  0.053053  0.044361   
answer3     0.004746  0.058809  0.079733  0.077824  0.012427  0.036891   
answer4     0.004306  0.041482  0.039216  0.105106  0.044102  0.017800   
answer5     0.005634  0.023264  0.044154  0.173628  0.100351  0.047550   
answer6     0.004848  0.062154  0.145386  0.077470  0.077592  0.030245   
answer7     0.004615  0.048639  0.050766  0.021128  0.062392  0.047385   
answer8     0.004237  0.023793  0.075360  0.030263  0.015480  0.060184   
answer9     0.003143  0.114966  0.032810  0.093566  0.095403  0.020552   
answer10    0.005723  0.067025  0.041729  0.044950  0.074137  0.010942   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.000111  0.070594  0.192128  0.020881  0.011049  0.010171   
answer2     0.001567  0.040803  0.041899  0.076615  0.061534  0.003444   
answer3     0.003622  0.107942  0.089565  0.045522  0.047487  0.006420   
answer4     0.000828  0.017979  0.077565  0.036964  0.007114  0.045434   
answer5     0.002767  0.029174  0.046819  0.279590  0.112444  0.117113   
answer6     0.000761  0.060397  0.062372  0.039840  0.049416  0.003732   
answer7     0.000446  0.076735  0.101710  0.058665  0.131804  0.020626   
answer8     0.002253  0.044640  0.113288  0.031472  0.034122  0.016224   
answer9     0.003370  0.148515  0.046542  0.166600  0.248802  0.151953   
answer10    0.003738  0.030123  0.156975  0.076413  0.064367  0.017975   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.041856  0.099661  0.102328  0.055707  0.038468  0.255886   
answer2     0.066032  0.082934  0.113261  0.110428  0.062818  0.157084   
answer3     0.074088  0.092625  0.123940  0.080122  0.062342  0.181359   
answer4     0.046539  0.093189  0.152933  0.066467  0.058091  0.128479   
answer5     0.058235  0.088280  0.098797  0.078218  0.072733  0.166673   
answer6     0.028715  0.061520  0.039668  0.119946  0.116825  0.158639   
answer7          NaN       NaN       NaN       NaN       NaN       NaN   
answer8     0.062074  0.150518  0.112111  0.079800  0.115703  0.104627   
answer9     0.015854  0.072840  0.081315  0.081876  0.054245  0.031263   
answer10    0.020723  0.078191  0.148434  0.078574  0.056461  0.251042   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [40]:
# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # 定数列を追加
    X_train_combined = sm.add_constant(X_train_combined)
    X_test = sm.add_constant(X_test)
    
    # ロジスティック回帰モデルの作成と訓練
    model = sm.Logit(y_train_combined, X_train_combined)
    result = model.fit(disp=False)
    print(result.summary())
    
    # 予測（確率）
    y_pred_prob = result.predict(X_test)
    
    # 確率を0または1に変換
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # 精度計算
    correct_predictions = (y_pred == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100  # 予測率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1735
Time:                        00:13:48   Log-Likelihood:                -147.89
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 4.711e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9089      0.489     -1.858      0.063      -1.868       0.050
AU01          -1.3632      4.625     -0.295      0.768     -10.427       7.701
AU02           2.3327

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  269
Model:                          Logit   Df Residuals:                      251
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1645
Time:                        00:13:49   Log-Likelihood:                -146.94
converged:                       True   LL-Null:                       -175.86
Covariance Type:            nonrobust   LLR p-value:                 2.367e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.1995      0.493     -2.431      0.015      -2.166      -0.233
AU01          -0.8009      4.582     -0.175      0.861      -9.781       8.179
AU02           2.5025      3.429      0.730      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1678
Time:                        00:13:49   Log-Likelihood:                -148.91
converged:                       True   LL-Null:                       -178.94
Covariance Type:            nonrobust   LLR p-value:                 1.032e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0636      0.491     -2.166      0.030      -2.026      -0.101
AU01          -1.6930      4.593     -0.369      0.712     -10.696       7.310
AU02           3.1289      3.445      0.908      0.3

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  267
Model:                          Logit   Df Residuals:                      249
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1972
Time:                        00:13:50   Log-Likelihood:                -139.54
converged:                       True   LL-Null:                       -173.81
Covariance Type:            nonrobust   LLR p-value:                 3.850e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0319      0.504     -2.046      0.041      -2.020      -0.043
AU01          -0.4292      4.841     -0.089      0.929      -9.918       9.060
AU02           1.1869      3.672      0.323      0.7

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      252
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1820
Time:                        00:13:50   Log-Likelihood:                -147.59
converged:                       True   LL-Null:                       -180.43
Covariance Type:            nonrobust   LLR p-value:                 1.186e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9798      0.483     -2.027      0.043      -1.927      -0.032
AU01          -1.5825      4.609     -0.343      0.731     -10.617       7.452
AU02           2.5599      3.433      0.746      0.4

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  270
Model:                          Logit   Df Residuals:                      252
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1692
Time:                        00:13:51   Log-Likelihood:                -149.90
converged:                       True   LL-Null:                       -180.43
Covariance Type:            nonrobust   LLR p-value:                 7.011e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0759      0.484     -2.222      0.026      -2.025      -0.127
AU01          -0.8760      4.535     -0.193      0.847      -9.764       8.012
AU02           2.2087      3.372      0.655      0.5

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                  268
Model:                          Logit   Df Residuals:                      250
Method:                           MLE   Df Model:                           17
Date:                Tue, 07 Jan 2025   Pseudo R-squ.:                  0.1849
Time:                        00:13:51   Log-Likelihood:                -146.26
converged:                       True   LL-Null:                       -179.44
Covariance Type:            nonrobust   LLR p-value:                 9.049e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9760      0.489     -1.995      0.046      -1.935      -0.017
AU01           1.0213      4.654      0.219      0.826      -8.100      10.142
AU02           1.6296      3.419      0.477      0.6

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],   # 正則化強度
    'penalty': ['l2'],              # L2正則化のみを使用（L1やElasticNetを使う場合はsolverを変更）
    'solver': ['lbfgs']             # lbfgsはL2正則化に対応
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # ロジスティック回帰モデルのグリッドサーチ
    model = LogisticRegression(max_iter=1000)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob.tolist()}")

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3152367648509745, 0.3322660497950206, 0.17121937379930455, 0.3493593645072179, 0.4723415785799884, 0.4210327592401845, 0.28639408010572287, 0.49326703782653797, 0.3178536593365929, 0.3305622080581996]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
クラス1の確率: [0.3071144711923794, 0.4331413047852056, 0.4949732915699083, 0.33988833153799425, 0.6777232688499163, 0.41924179584520294, 0.42946076872282823, 0.38307903749117217, 0.35604023694565323, 0.43708820539996684]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 

最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID54_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36150514798610844, 0.360753135012148, 0.3628503179702128, 0.36166650100652187, 0.3614030473947944, 0.3606508533218448, 0.3611201282299482, 0.361617227653864]

===== テストデータ: ファイル 23/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID56_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36245512742797004, 0.3606532170376084, 0.3608893044874888, 0.36126198912953444, 0.36106977594854783, 0.3610159588824907, 0.3606762079422699, 0.3609972893062906]

===== テストデータ: ファイル 24/31 =====
最適パラメータ: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
ファイル名: filtered_ID57_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35641048155225885, 0.355444296897294, 0.3560031061629417, 0.35583680282